# Visualizing opertions graph

[From original documentation:](https://www.tensorflow.org/guide/intro_to_graphs#:~:text=TensorFlow%20uses%20graphs%20as%20the,(%22constant%20folding%22).)

>Graphs are data structures that contain a set of tf.Operation objects, which represent units of computation; and tf.Tensor objects, which represent the units of data that flow between operations.

Tensorboard provides us with tools to visualize the computation graph.

Keras-based models (which use `model.fit(...)` method) use [TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard). In this case, we only have to pass
```
    tf.keras.callbacks.TensorBoard(..., write_graph=True, ...)
```

In [1]:
import tensorflow as tf 

from toy_model import create_model
from tensorflow.keras.callbacks import TensorBoard

2022-05-23 13:00:29.304987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 13:00:29.305006: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


we will use the same data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
n_classes = tf.unique(y_test).y.shape[0]

2022-05-23 13:00:30.725728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-23 13:00:30.725752: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-23 13:00:30.725765: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (appa): /proc/driver/nvidia/version does not exist
2022-05-23 13:00:30.726081: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


and the previous classifier,

In [4]:
clf = create_model(n_classes=n_classes)

when you compile keras-based models the graph is generated.

In [21]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
clf.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

tb_cback = TensorBoard(log_dir='logs', write_graph=True, update_freq='epoch')

However, the graph is not sent to Tensorboard until `fit(callbacks=[tb_cback])` is executed. 

In [22]:
x_train_net = tf.expand_dims(x_train, axis=3)
y_train_net = tf.one_hot(y_train, n_classes)

x_train_net.shape, y_train_net.shape

(TensorShape([60000, 28, 28, 1]), TensorShape([60000, 10]))

In [23]:
%%time
history = clf.fit(x_train_net, 
                  y_train_net, 
                  batch_size=256, 
                  epochs=1, 
                  validation_split=0.2,
                  callbacks=[tb_cback])

188/188 [==============================] - 8s 43ms/step - loss: 1.7399 - accuracy: 0.8645 - val_loss: 0.1924 - val_accuracy: 0.9575
CPU times: user 49.7 s, sys: 1.52 s, total: 51.2 s
Wall time: 8.49 s


In [11]:
%load_ext tensorboard

In [25]:
%tensorboard --logdir logs

## Custom training loops

If you are not using the `model.fit(...)` function then you have to register the graph to Tensorboard. This scenario provides us more flexibity to control the graph structure.

To show this feature we will create a custom layer and we will visualize it

In [25]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs, dropout=0.5, name=''):
        super(MyDenseLayer, self).__init__(name=name)
        self.num_outputs = num_outputs
        self.bn_0 = tf.keras.layers.LayerNormalization(name='BN')
        self.bn_1 = tf.keras.layers.LayerNormalization(name='BN')
        self.drop = tf.keras.layers.Dropout(dropout, name='Dropout')
        self.layer_0 = tf.keras.layers.Dense(self.num_outputs, name='layer_0')
        self.layer_1 = tf.keras.layers.Dense(self.num_outputs, name='layer_1')
        
    def call(self, inputs):
        with tf.name_scope("First_Part") as scope0:
            x = self.layer_0(inputs)
            x = self.bn_0(x)
            x = self.drop(x)
        
        with tf.name_scope("Second_Part") as scope1:
            y = self.layer_1(inputs)
            y = self.bn_1(y)
        
        return x, y

In [26]:
inputs = tf.keras.Input(shape=(4, ), name="serie")
x, y = MyDenseLayer(num_outputs=100, name='MyLayer')(inputs)
outputs = (x, y)
model = tf.keras.Model(inputs, outputs, name="toy_model")

Now we create our graph writer

In [27]:
from tensorflow.summary import create_file_writer

log_dir = './custom_graph'
train_writer = create_file_writer('{}/train'.format(log_dir))

Graphs can only be created using `tf.function` decorator which optimizes our code and creates the computational graph automatically.

In [28]:
def draw_graph(model, dataset, writer, logdir=''):
    '''Decorator that reports store fn graph.'''

    @tf.function
    def fn(x):            
        x = model(x)

    tf.summary.trace_on(graph=True, profiler=False)
    fn(dataset)
    with writer.as_default():
        tf.summary.trace_export(
            name='model',
            step=0,
            profiler_outdir=logdir)

In [29]:
dataset = tf.random.normal([2, 4])

In [30]:
draw_graph(model, dataset, train_writer)

In [31]:
%tensorboard --logdir custom_graph/